<a href="https://colab.research.google.com/github/gionata/optimization-notebooks/blob/main/Production_mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install dependencies
%pip install -q amplpy

In [12]:
from amplpy import AMPL, ampl_notebook
from google.colab import userdata # for accessing colab secrets

ampl = ampl_notebook(
    modules=["cbc"],
    license_uuid=userdata.get('license'),  # license from "secret"
)   # instantiate AMPL object and register magics

Licensed to AMPL Community Edition License for <gionata.massi@gmail.com>.


In [ ]:
%%writefile product_mix.mod
# file: product_mix.mod
#
# Product mix problem
#
# Maximize profit subject to less than or equal to constraints

# Sets
set Products;
set Resources;

# Decision variables
var x {j in Products} >= 0;

# Parameters
param profit {j in Products} >= 0;
param availability {i in Resources} >= 0;
param consumption {i in Resources, j in Products} >= 0;

# Objective function
maximize total_profit: sum {j in Products} profit[j]*x[j];

# Constraints
s.t.   usage {i in Resources}:
	sum {j in Products} consumption[i, j]*x[j]
		<=  availability[i];


Writing product_mix.mod


In [ ]:
%%writefile product_mix.dat
# file: product_mix.dat

set Products := prod_1 prod_2;
set Resources := raw_material labour demand;
param profit := prod_1 120 prod_2 40;
param availability :=
	raw_material 2200
	labour        320
	demand        100;
param consumption: prod_1 prod_2 :=
	raw_material 40 20
	labour        8  2
	demand        1  1;


Writing product_mix.dat


In [ ]:
%%ampl_eval
# file: product_mix.run

model product_mix.mod;
data product_mix.dat;
show;
expand;
option solver cbc;
solve;
display total_profit;
display x;
display usage;



parameters:   availability   consumption   profit

sets:   Products   Resources

variable:   x

constraint:   usage

objective:   total_profit
maximize total_profit:
	120*x['prod_1'] + 40*x['prod_2'];

subject to usage['raw_material']:
	40*x['prod_1'] + 20*x['prod_2'] <= 2200;

subject to usage['labour']:
	8*x['prod_1'] + 2*x['prod_2'] <= 320;

subject to usage['demand']:
	x['prod_1'] + x['prod_2'] <= 100;

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 5400
0 simplex iterations
total_profit = 5400

x [*] :=
prod_1  25
prod_2  60
;

usage [*] :=
      demand   0
      labour  10
raw_material   1
;

